In [1]:
import os
import time
import win32api
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Define the base directory where you want to save the files
# base_directory = "C:/path/to/your/directory"  # Change this to your desired path



# Create a WebDriver instance (in this example, using Chrome)
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# URL you want to open
url = "https://www.sspcrs.ie/portal/annual-reporting/report/pharmacy"

# Open the URL
driver.get(url)
time.sleep(10)


reportPage = driver.find_element(By.XPATH, "/html/body/div/div/section/section[2]/section/div/div[4]/div[2]/h3/form/button") #xpath for Scraping top 100 medication by cost


reportPage.click()
time.sleep(10)

firstList= ['GMS']
secondList= ['201601','201602','201603','201604','201605','201606','201607','201608','201609','201610','201611','201612', 
             '201701','201702','201703','201704','201705','201706','201707','201708','201709','201710','201711','201712',
             '201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
             '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
             '202001','202002','202003','202004','202005','202006','202007','202008','202009','202010','202011','202012',
             '202101','202102','202103','202104','202105','202106','202107','202108','202109','202110','202111','202112',
             '202201','202202','202203','202204','202205','202206','202207','202208','202209','202210','202211','202212',
             '202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202312','202401','202402','202403','202404','202405','202406','202407','202408','202409','202410']

# Get the arrow on the schema list
schemaElement = driver.find_element(By.XPATH, "//div[@class='promptChoiceListBox']/img")
schemaElement.click()

time.sleep(3)
# Loop over the schema list
for sch in firstList:
    time.sleep(5)
    wait = WebDriverWait(driver, 10)

    # Select the schema from the first comboBox
    schemaChoice = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@title='"+sch+"']")))
    schemaChoice.click()

    # Click away to validate the schema selection
    td_element = driver.find_element(By.CSS_SELECTOR, "td.TitleLogo")
    td_element.click()

    time.sleep(5)

    # Create a directory for the schema if it doesn't exist
    schema_directory = os.path.join(base_directory, sch)
    if not os.path.exists(schema_directory):
        os.makedirs(schema_directory)

    # Loop over the month list
    for month in secondList:
        # Click on month list
        monthList = driver.find_element(By.XPATH, "//td[@class='data promptControl']//img[contains(@class, 'promptComboBoxButtonMoz')]")
        monthList.click()
        time.sleep(5)
        wait = WebDriverWait(driver, 10)
        # Click on the month
        monthChoice = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@title='"+month+"']")))
        monthChoice.click()
        time.sleep(3)
        monthList.click()
        time.sleep(3)
        monthList.click()
        time.sleep(3)

        # Click on apply button
        applyButton = driver.find_element(By.XPATH, "//*[@id='gobtn']")
        applyButton.click()
        
        # Put the mouse cursor away
        win32api.SetCursorPos((100,100))

        time.sleep(10)
        # Find the table element
        table_element = driver.find_element(By.XPATH, "//table[contains(@id, 'saw_')]")

        # Extract the table data
        table_data = table_element.get_attribute("outerHTML")
        

        # Parse the HTML table using Pandas
        df = pd.read_html(table_data, header=None, skiprows=9)[0]

        # Add a new column month value at the end
        df["Month"] = month
        
        # Rename columns for the individual file
        df.columns = ["Position", "Name", "Ingredient Cost €", "% Ingredient Cost", "Prescribing Frequency", "% Prescribing Frequency", "mth_yr"]
        
        # Save the table data (with headers) to an Excel file
        file_path = os.path.join(schema_directory, sch + "_" + month + ".xlsx")
        df.to_excel(file_path, header=True, index=False)
        print(f"Downloaded and saved data for {sch} {month} to {file_path}")
        time.sleep(5)
        
        # Move mouse cursor
        win32api.SetCursorPos((110,110))
        # End of month loop

    # Put all the reports on one list
    schemaexportlist = []
    for month in secondList:
        file_path = os.path.join(schema_directory, sch + "_" + month + ".xlsx")
        df = pd.read_excel(file_path, header=0)
        df.columns = ["Position", "Name", "Ingredient Cost €", "% Ingredient Cost", "Prescribing Frequency", "% Prescribing Frequency", "mth_yr"]
        schemaexportlist.append(df)
        
    # Use pandas concat to create one Excel file
    final_file_path = os.path.join(base_directory, sch + '.xlsx')
    df = pd.concat(schemaexportlist, ignore_index=True)
    
    # Modify the final dataframe as per requirements
    df = df.drop(columns=["Position"])
    df = df.rename(columns={
        "Name": "name",
        "Prescribing Frequency": "Prescribing_Frequency",
        "% Prescribing Frequency": "% Prescribing Frequency (of Scheme Total Frequency)",
        "Ingredient Cost €": "Ingredient Cost €",
        "% Ingredient Cost": "% Ingredient Cost (of Scheme Total Cost)"
    })

    # Remove '%' sign from percentage columns
    df["% Prescribing Frequency (of Scheme Total Frequency)"] = df["% Prescribing Frequency (of Scheme Total Frequency)"].str.rstrip('%').astype(float)
    df["% Ingredient Cost (of Scheme Total Cost)"] = df["% Ingredient Cost (of Scheme Total Cost)"].str.rstrip('%').astype(float)

    df["Type"] = "Medication"
    df["Scheme"] = "GMS"
    df["Filter_scrabe"] = "Cost"
    df["Prescribing Unit Cost €"] = ""
    df["dispensing_rate_per_1000"] = ""
    df["cost_rate_per_1000"] = ""
    
    df = df[[
        "name", "Prescribing_Frequency", "% Prescribing Frequency (of Scheme Total Frequency)",
        "Ingredient Cost €", "% Ingredient Cost (of Scheme Total Cost)", "Prescribing Unit Cost €",
        "dispensing_rate_per_1000", "cost_rate_per_1000", "Scheme", "mth_yr", "Type", "Filter_scrabe"
    ]]
    
    df.to_excel(final_file_path, index=False)
    print(f"Processed and saved combined data for {sch} to {final_file_path}")

# Close the WebDriver instance
driver.quit()


Downloaded and saved data for GMS 202403 to C:/ahmedhassanali/PCRS data/shiny/Stage 1 web scraping/042024/medsbycost\GMS\GMS_202403.xlsx
Downloaded and saved data for GMS 202404 to C:/ahmedhassanali/PCRS data/shiny/Stage 1 web scraping/042024/medsbycost\GMS\GMS_202404.xlsx
Processed and saved combined data for GMS to C:/ahmedhassanali/PCRS data/shiny/Stage 1 web scraping/042024/medsbycost\GMS.xlsx


In [2]:
import pandas as pd

# Load the entire data from the first Excel file

#df1 = pd.read_excel('the pathh to the file /medsbycost/GMS.xlsx') # Change this to your desired path
# Define the paths for the Excel files
df1_path = r'C:\project\RxTrends-RShiny 2025\newdata\bycost\merged_data.xlsx'  # Path to the first file
df2_path = r'C:\project\RxTrends-RShiny 2025\newdata\ATC codes for common medications.xlsx'  # Path to the second file
output_path = r'C:\project\RxTrends-RShiny 2025\newdata\byfreq\medsbycost_code.xlsx'  # Path for the output file

# Load the relevant columns from the second Excel file
df2 = pd.read_excel('C:/ / /ATC codes for common medications.xlsx', usecols=['Drug', 'ATC code']) # Change this to the file path

# Merge the full df1 with df2 on the medication names, without losing any rows from df1
merged_df = pd.merge(df1, df2, left_on='name', right_on='Drug', how='left')

# Create a new column that combines the name and ATC code only for 'Medication' types
merged_df['name(ATC code)'] = merged_df.apply(
    lambda x: f"{x['name']} ({x['ATC code']})" if x['Type'] == 'Medication' and pd.notna(x['ATC code']) else x['name'], 
    axis=1
)

# Insert the new column right after the 'name' column
# Find the index of the 'name' column
name_idx = merged_df.columns.get_loc('name')
# Insert the new column after the 'name' column
merged_df.insert(name_idx + 1, 'Name with ATC Code', merged_df.pop('name(ATC code)'))

# Drop the 'Drug' column as it's no longer needed
merged_df.drop(columns='Drug', inplace=True)

## Save the updated DataFrame back to an Excel file

merged_df.to_excel('the path to the file /medsbycost_code.xlsx', index=False) # Change this to your desired path

print("File has been updated and saved")


File has been updated and saved
